In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
import pandas as pd

In [2]:
df = pd.read_csv("../../data/naver_shopping_tokenized.csv", encoding = 'utf-8', index_col = 0)

In [3]:
from sklearn.model_selection import train_test_split
y = df['y'].values.tolist()
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['text'], y, test_size = 0.2, random_state = 0)

In [4]:
okt = Okt()

def okt_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    
    return tokens_ko

## DTM 기준, 주요 긍정 단어와 주요 부정 단어 추출

In [5]:
vect = CountVectorizer(tokenizer = okt_tokenizer, min_df = 3, ngram_range=(1,2))
X_train_tf = vect.fit_transform(X_train_texts)
X_test_tf = vect.transform(X_test_texts)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

In [19]:
# 단어들을 번호 기준 내림차순으로 저장
vocablist = [word for word, number in sorted(vect.vocabulary_.items(), key = lambda x:x[1])]

In [14]:
from sklearn.linear_model import LogisticRegression
lr_DTM = LogisticRegression(random_state = 0, solver = 'liblinear', C = 1)
lr_DTM.fit(X_train_tf, y_train)
y_pred_DTM = lr_DTM.predict(X_test_tf)

In [11]:
coefficients = lr_DTM.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('▶긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)◀')
for word_num, coef in sorted_coefficients[:10]:
    print('{0:}({1:.3f})'.format(vocablist[word_num], coef))

print('\n▶부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)◀')
for word_num, coef in sorted_coefficients[-10:][::-1]: 
    print('{0:}({1:.3f})'.format(vocablist[word_num], coef))

긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)
맛있습니다(2.755)
맛있네요(2.703)
이쁩니다(2.661)
맛있고(2.497)
맛있어요(2.496)
만족스럽습니다(2.479)
튼튼합니다(2.303)
잘 왔어요(2.270)
최고(2.252)
좋았습니다(2.238)

부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)
최악(-3.713)
실망(-2.971)
좋네요 좋네요(-2.902)
비추(-2.780)
맛있어요 맛있어요(-2.680)
별루(-2.674)
실망했어요(-2.640)
만족합니다 만족합니다(-2.617)
불편해요(-2.610)
굿 ㅎ(-2.492)


## TF-IDF 기준, 주요 긍정 단어와 주요 부정 단어 추출

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.9, tokenizer = okt_tokenizer)
tfidf_vectorizer.fit(X_train_texts)
tfidf_matrix_train = tfidf_vectorizer.transform(X_train_texts)
tfidf_matrix_test = tfidf_vectorizer.transform(X_test_texts)

In [18]:
vocablist2 = [word for word, number in sorted(tfidf_vectorizer.vocabulary_.items(), key = lambda x:x[1])]

In [15]:
lr_tfidf = LogisticRegression(random_state = 0, solver = 'liblinear', C = 3.5)
lr_tfidf.fit(tfidf_matrix_train, y_train)

LogisticRegression(C=3.5, random_state=0, solver='liblinear')

In [20]:
coefficients2 = lr_tfidf.coef_.tolist()

sorted_coefficients2 = sorted(enumerate(coefficients2[0]), key = lambda x:x[1], reverse = True)


print('▶긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)◀')
for word_num, coef in sorted_coefficients2[:10]:
    print('{0:}({1:.3f})'.format(vocablist2[word_num], coef))

print('\n▶부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)◀')
for word_num, coef in sorted_coefficients2[-10:][::-1]: 
    print('{0:}({1:.3f})'.format(vocablist2[word_num], coef))

긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)
맛있어요(8.237)
좋네요(7.898)
좋고(7.572)
좋아요(7.396)
좋습니다(7.313)
예뻐요(6.687)
깔끔하고(6.643)
맛있네요(6.573)
빠르고(6.478)
걱정했는데(6.396)

부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)
별로에요(-10.027)
별로네요(-9.625)
실망입니다(-8.191)
별로입니다(-7.532)
별로예요(-7.155)
불편해요(-7.002)
실망(-6.814)
다시는(-6.634)
최악(-6.547)
버렸어요(-6.546)
